# North America virtual geomagnetic poles

This notebook plots the compiled VGP data.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pmagpy.ipmag as ipmag
import pmagpy.pmag as pmag
import cartopy.crs as ccrs
import seaborn as sns
%config InlineBackend.figure_format='retina'

In [5]:
current_path = os.getcwd() 
data_path = current_path + "/data/"
file_name = "compiled_vgps.csv"
vgps = pd.read_csv(data_path + file_name)
vgps.head()

,Unnamed: 0,pole_id,vgp_id,Name,slat,slon,n,dec,inc,k,...,vgp_lat,vgp_lon,vgp_lat_rev,vgp_lon_rev,vgp_lat_SH,vgp_lon_SH,comments on age,dip_dir,dip,comments (x = excluded)
0,0,VGPs_NAM\1,1,5L209,NaN,NaN,5.0,338.0,37.6,234.0,...,64.8,116.3,NaN,NaN,-64.8,296.3,NaN,NaN,NaN,NaN
1,1,VGPs_NAM\1,2,5L233,NaN,NaN,8.0,332.3,37.9,1604.0,...,61.0,124.6,NaN,NaN,-61.0,304.6,NaN,NaN,NaN,NaN
2,2,VGPs_NAM\1,3,5L352,NaN,NaN,8.0,17.7,25.8,741.0,...,61.3,23.0,NaN,NaN,-61.3,203.0,NaN,NaN,NaN,NaN
3,3,VGPs_NAM\1,4,5L344,NaN,NaN,8.0,5.8,48.4,471.0,...,80.5,29.0,NaN,NaN,-80.5,209.0,NaN,NaN,NaN,NaN
4,4,VGPs_NAM\1,5,5L334,NaN,NaN,5.0,358.9,58.5,411.0,...,88.3,212.0,NaN,NaN,-88.3,32.0,NaN,NaN,NaN,NaN


In [9]:
current_path = os.getcwd() 
data_path_PP = current_path + "/data/poles_NAM"
file_name = "/NAM_poles_0_130.xlsx"
poles = pd.read_excel(data_path_PP + file_name)
poles['ID'] = poles['pole_id']
poles.head()

,pole_id,name,min_age,max_age,age,slat,slon,N,k,alpha95,...,Rlat,Rlon,Euler_lat,Euler_lon,Euler_ang,author,Q,reliability,age constraints,ID
0,1,USGS SW North America composite,0.0,1.0,0.5,36.30,242.13,38,24.7,4.8,...,-86.3,5.7,0.0,0.0,0.0,Mankinen (2008),5,NaN,Brunhes normal chron,1
1,2,Michoacan Guanajuato volcanic ﬁeld,0.0,0.6,0.3,19.25,257.67,8,29.0,9.1,...,-86.4,9.2,79.2,23.0,0.2,Maciel Peña et al. (2009),5,NaN,Ar-Ar ages of 82 to 612 ka (from another study),2
2,3,Tequila volcanic ﬁelds,0.1,1.1,0.6,20.46,256.76,17,26.0,7.1,...,-85.3,-94.1,79.9,22.7,0.3,Ceja et al. (2006),5,NaN,Ar/Ar ages of 0.115 to 1.13 Ma (from another s...,3
3,4,Trans Mexican Volcanic Belt,0.0,2.0,1.0,19.40,260.10,24,47.3,4.3,...,-88.9,-75.1,79.9,22.7,0.3,Ruiz-Martínez et al. (2010),5,NaN,all Quaternary rocks (some K-Ar dated),4
4,5,Katherine Creek sediments,0.0,3.0,1.5,65.00,232.40,100,9.1,4.9,...,-80.0,4.2,80.8,22.8,0.4,Barendregt et al. (1996),3,NaN,magnetostratigraphy,5


In [10]:
def vgp_pole_means(study_id,
                   print_compilation_pole=True,
                   print_vgp_calc_pole=True,
                   plot_vgp_calc_pole=True,
                   plot_compilation_pole=True):
    
    compilation_pole = poles[poles.ID == study_id]
    site_vpgs = vgps[vgps.pole_id == study_id]
    
    #make it so that all VGPs are northern hemisphere
    vgp_lats = []
    vgp_lons = []
    for i in site_vpgs.index:
        if site_vpgs['vgp_lat'][i]>0:
            vgp_lons.append(site_vpgs.vgp_lon[i])
            vgp_lats.append(site_vpgs.vgp_lat[i])
        if site_vpgs['vgp_lat'][i]<0:
            vgp_lons.append((site_vpgs.vgp_lon[i]+180)%360)
            vgp_lats.append(-site_vpgs.vgp_lat[i])
    
    if len(vgps[vgps.pole_id == study_id]) > 0:
        pole_from_vgps = ipmag.fisher_mean(vgp_lons,vgp_lats)
    
    if print_compilation_pole:
        print('Pole name: ' + str(compilation_pole['name'].tolist()[0]))
        print('')
        print('South pole within compilation')
        print('Plon: ' + str(compilation_pole['plon'].tolist()[0]) +
              '  Plat: ' + str(compilation_pole['plat'].tolist()[0]))
        print('Number of directions in mean (n): ' + str(compilation_pole['N'].tolist()[0]))
        print('Angular radius of 95% confidence (A_95): ' +
              str(compilation_pole['A95'].tolist()[0]))
        print('Precision parameter (k) estimate: ' +
              str(compilation_pole['K'].tolist()[0]))
        print('')
        print('North pole within compilation')
        print('Plon: ' + str((compilation_pole['plon'].tolist()[0]+180)%360) +
              '  Plat: ' + str(-compilation_pole['plat'].tolist()[0]))
        print('Number of directions in mean (n): ' + str(compilation_pole['N'].tolist()[0]))
        print('Angular radius of 95% confidence (A_95): ' +
              str(compilation_pole['A95'].tolist()[0]))
        print('Precision parameter (k) estimate: ' +
              str(compilation_pole['K'].tolist()[0]))
        
    if len(vgps[vgps.pole_id == study_id]) > 0:
    
        if print_vgp_calc_pole:
            print('')
            print('pole calculated from VGPs')
            ipmag.print_pole_mean(pole_from_vgps)

        map_axis = ipmag.make_orthographic_map(0, 90, land_color='white', 
                                               land_edge_color='grey',
                                               figsize=(7,7))
        ipmag.plot_vgp(map_axis, site_vpgs['vgp_lon'], site_vpgs['vgp_lat'],markersize=30,
                       color='darkblue',label='vgps')

        if plot_vgp_calc_pole:
            ipmag.plot_pole(map_axis, plon=pole_from_vgps['dec'], 
                            plat=pole_from_vgps['inc'], 
                            A95=pole_from_vgps['alpha95'], 
                            color = 'darkred', marker='s', markersize=40,
                            label='vgp calc pole')

        if plot_compilation_pole:
            if np.isnan(compilation_pole['A95'].tolist()[0]):
                A95 = 0
            else:
                A95=compilation_pole['A95'].tolist()[0]
            ipmag.plot_pole(map_axis, plon=(compilation_pole['plon'].tolist()[0]+180)%360, 
                            plat=-compilation_pole['plat'].tolist()[0], 
                            A95=A95, 
                            color = 'orange', marker='d', markersize=40,label='compilation pole')
        plt.legend()
        plt.show()

In [11]:
for n in range(1,40):
    vgp_pole_means(n)

Pole name: USGS SW North America composite

South pole within compilation
Plon: 5.7  Plat: -86.3
Number of directions in mean (n): 38
Angular radius of 95% confidence (A_95): nan
Precision parameter (k) estimate: nan

North pole within compilation
Plon: 185.7  Plat: 86.3
Number of directions in mean (n): 38
Angular radius of 95% confidence (A_95): nan
Precision parameter (k) estimate: nan
Pole name: Michoacan Guanajuato volcanic ﬁeld

South pole within compilation
Plon: 39.9  Plat: -84.4
Number of directions in mean (n): 8
Angular radius of 95% confidence (A_95): 8.5
Precision parameter (k) estimate: 33.0

North pole within compilation
Plon: 219.9  Plat: 84.4
Number of directions in mean (n): 8
Angular radius of 95% confidence (A_95): 8.5
Precision parameter (k) estimate: 33.0
Pole name: Tequila volcanic ﬁelds

South pole within compilation
Plon: 264.3  Plat: -85.8
Number of directions in mean (n): 17
Angular radius of 95% confidence (A_95): 6.9
Precision parameter (k) estimate: 27.5



In [ ]:
GCDs = []
ID = []
for _, i in poles_from_vgps.iterrows():
        
    pole_vgps = poles_from_vgps[poles_from_vgps['pole_id'] == str(i.pole_id)]
    pole_reference = df_NAM[df_NAM['pole_id'] == int(i.pole_id)]
    
    
    cart_ref = (spherical2cartesian([np.radians(pole_reference['plat'].values)[0],np.radians(pole_reference['plon'].values)[0]]))
    cart_vgps = (spherical2cartesian([np.radians(pole_vgps['inc'].values)[0], np.radians(pole_vgps['dec'].values)[0]]))
    
    GCDs.append(GCD_cartesian(cart_ref,cart_vgps)*180/np.pi)
    ID.append(int(i.pole_id))
    
# dictionary = {'id':ID, 'gcds': GCDs}
fig = plt.figure(figsize=(15,5))
plt.scatter(ID, GCDs)
plt.show()